In [34]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import zscore
import os



df = pd.read_csv('../../data/benin.csv', encoding =  'latin1')

C:\Users\hp\AppData\Local\Temp\ipykernel_13184\3481008962.py:9: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../data/benin.csv', encoding =  'latin1')


In [35]:
df.describe()


,Comments
count,0.0
mean,NaN
std,NaN
min,NaN
25%,NaN
50%,NaN
75%,NaN
max,NaN


In [36]:
# Convert Timestamp to datetime
df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')
# Convert all other columns (except Timestamp and Comments) to numeric
for col in df.columns:
    if col not in ['Timestamp', 'Comments']:
        df[col] = pd.to_numeric(df[col], errors='coerce')
df.info()

C:\Users\hp\AppData\Local\Temp\ipykernel_13184\3908643666.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['Timestamp'] = pd.to_datetime(df['Timestamp'], errors='coerce')


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 525601 entries, 0 to 525600
Data columns (total 19 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   Timestamp      525600 non-null  datetime64[ns]
 1   GHI            525600 non-null  float64       
 2   DNI            525600 non-null  float64       
 3   DHI            525600 non-null  float64       
 4   ModA           525600 non-null  float64       
 5   ModB           525600 non-null  float64       
 6   Tamb           525600 non-null  float64       
 7   RH             525600 non-null  float64       
 8   WS             525600 non-null  float64       
 9   WSgust         525600 non-null  float64       
 10  WSstdev        525600 non-null  float64       
 11  WD             525600 non-null  float64       
 12  WDstdev        525600 non-null  float64       
 13  BP             525600 non-null  float64       
 14  Cleaning       525600 non-null  float64       
 15  

In [ ]:
df.isna().sum()

Timestamp             1
GHI                   1
DNI                   1
DHI                   1
ModA                  1
ModB                  1
Tamb                  1
RH                    1
WS                    1
WSgust                1
WSstdev               1
WD                    1
WDstdev               1
BP                    1
Cleaning              1
Precipitation         1
TModA                 1
TModB                 1
Comments         525601
dtype: int64

In [38]:

missing_counts = df.isna().sum()
missing_percent = (missing_counts / len(df)) * 100
columns_with_gt_5pct_nulls = missing_percent[missing_percent > 5]
print("Columns with more than 5% missing values:")
print(columns_with_gt_5pct_nulls)


Columns with more than 5% missing values:
Comments    100.0
dtype: float64


In [39]:
# Define relevant columns
irradiance_cols = ['GHI', 'DNI', 'DHI']
sensor_cols = ['ModA', 'ModB']
wind_cols = ['WS', 'WSgust']
all_cols = irradiance_cols + sensor_cols + wind_cols

# --- 1. Convert columns to numeric (safe parsing)
for col in all_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')  # convert invalid strings to NaN

# --- 2. Check for missing values
missing = df[all_cols].isna().sum()
missing_percent = (missing / len(df)) * 100
print("🔍 Missing Values (>0):")
print(missing[missing > 0])

# --- 3. Basic descriptive statistics
print("\n📊 Descriptive Stats:")
print(df[all_cols].describe())

# --- 4. Check for negative values
print("\n❗ Negative or unrealistic values:")
for col in all_cols:
    num_negatives = df[df[col] < 0].shape[0]
    print(f"{col} has {num_negatives} negative values")

# --- 5. Detect outliers using IQR method
print("\n🚨 Outlier detection using IQR:")
for col in all_cols:
    Q1 = df[col].quantile(0.25)
    Q3 = df[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    outliers = df[(df[col] < lower_bound) | (df[col] > upper_bound)]
    print(f"{col}: {len(outliers)} outliers")



🔍 Missing Values (>0):
GHI       1
DNI       1
DHI       1
ModA      1
ModB      1
WS        1
WSgust    1
dtype: int64

📊 Descriptive Stats:
                 GHI            DNI            DHI           ModA  \
count  525600.000000  525600.000000  525600.000000  525600.000000   
mean      240.559452     167.187516     115.358961     236.589496   
std       331.131327     261.710501     158.691074     326.894859   
min       -12.900000      -7.800000     -12.600000       0.000000   
25%        -2.000000      -0.500000      -2.100000       0.000000   
50%         1.800000      -0.100000       1.600000       4.500000   
75%       483.400000     314.200000     216.300000     463.700000   
max      1413.000000     952.300000     759.200000    1342.300000   

                ModB             WS         WSgust  
count  525600.000000  525600.000000  525600.000000  
mean      228.883576       2.121113       2.809195  
std       316.536515       1.603466       2.029120  
min         0.000000    

In [40]:


# Define target columns
columns = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']

# Ensure all columns are numeric
for col in columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Drop rows with missing values in selected columns (optional but recommended for Z-score)
df_clean = df[columns].dropna()

# Compute Z-scores
z_scores = df_clean.apply(zscore)

# Flag rows where any Z-score > 3 or < -3
outlier_flags = (z_scores.abs() > 3)

# Combine with original indices
outlier_rows = outlier_flags.any(axis=1)

# Print number of outliers and sample
print(f"\n🚨 Total rows with |Z| > 3 in any column: {outlier_rows.sum()}")
print("\n🔍 Sample outlier rows:")
print(df_clean[outlier_rows].head())

# Optional: Get index of flagged rows
outlier_indices = df_clean[outlier_rows].index



🚨 Total rows with |Z| > 3 in any column: 7740

🔍 Sample outlier rows:
        GHI    DNI    DHI    ModA    ModB   WS  WSgust
671   836.0  235.0  610.6   778.8   783.8  3.2     4.1
672  1274.0  698.8  615.2  1210.3  1210.3  3.4     4.1
673   938.0  340.1  612.8   891.1   891.1  3.4     4.6
674   718.5  126.8  593.2   682.6   682.6  4.7     5.6
675  1349.0  771.8  618.0  1281.5  1281.5  4.0     4.6


In [41]:
# Define key columns
key_cols = ['GHI', 'DNI', 'DHI', 'ModA', 'ModB', 'WS', 'WSgust']

# Convert to numeric safely
for col in key_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')

# Impute missing values with median
for col in key_cols:
    median = df[col].median()
    df[col].fillna(median, inplace=True)


C:\Users\hp\AppData\Local\Temp\ipykernel_13184\774207399.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(median, inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_13184\774207399.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing '

In [42]:


os.makedirs("data", exist_ok=True)

# Replace 'Kenya' with the actual country name if needed
country = "Benin"
df.to_csv(f"data/{country}_clean.csv", index=False)
